In [ ]:
import pandas as pd
import os
import numpy as np
import gc
import torch
import warnings
warnings.filterwarnings('ignore')

# Sklearn imports
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import  statsmodels.api as sm

# Keras imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import backend as K

# Scikeras wrapper for Keras
from scikeras.wrappers import KerasRegressor

# LightGBM
import lightgbm as lgb

# (Hypothetical) TabPFN Regressor
# If the TabPFN package does not provide a regressor, remove or replace this import
from tabpfn import TabPFNRegressor  # Placeholder for a potential TabPFNRegressor

In [ ]:


###############################################################################
# MLP Model Definition
###############################################################################
def create_mlp_model(input_shape):
    """
    Create a simple MLP model for regression.
    """
    model = Sequential([
        Dense(1024, activation="relu", input_shape=(input_shape,)),
        Dropout(0.3),
        Dense(512, activation="relu"),
        Dropout(0.3),
        Dense(256, activation="relu"),
        Dropout(0.3),
        Dense(128, activation="relu"),
        Dropout(0.3),
        # Final layer for regression: linear activation, 1 output
        Dense(1, activation="linear")
    ])
    model.compile(optimizer='adam', 
                  loss='mean_squared_error', 
                  metrics=['mean_absolute_error'])
    return model

###############################################################################
# CUDA / Memory Cleanup
###############################################################################
def clean_up_cuda(model):
    """
    Free up GPU memory and clear Keras session.
    """
    # Delete the Keras model
    K.clear_session()
    del model
    
    # Run garbage collection
    gc.collect()
    
    # Free CUDA memory
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    
    print("CUDA memory cleared and model deleted.")

###############################################################################
# Regression Metrics & Aggregation
###############################################################################
def evaluate_regression_performance(y_true, y_pred):
    """
    Compute regression metrics for predictions.
    Returns a dictionary with MSE, MAE, and R2.
    """
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    results = {
        'mse': mse,
        'mae': mae,
        'r2': r2
    }
    return results

def print_regression_performance(results):
    """
    Print regression performance metrics nicely.
    """
    print(f"MSE: {results['mse']:.4f}")
    print(f"MAE: {results['mae']:.4f}")
    print(f"R²:  {results['r2']:.4f}")

def aggregate_cv_metrics_and_print(all_results, model_name, tag="Validation"):
    """
    Aggregate cross-validation metrics (MSE, MAE, R2)
    and return mean + std across folds.
    """
    aggregated = {
        'mse': [],
        'mae': [],
        'r2': []
    }
    
    for result in all_results:
        aggregated['mse'].append(result['mse'])
        aggregated['mae'].append(result['mae'])
        aggregated['r2'].append(result['r2'])
        
    summary = {
        'mean_mse':  np.mean(aggregated['mse']),
        'std_mse':   np.std(aggregated['mse']),
        'mean_mae':  np.mean(aggregated['mae']),
        'std_mae':   np.std(aggregated['mae']),
        'mean_r2':   np.mean(aggregated['r2']),
        'std_r2':    np.std(aggregated['r2']),
    }
    print(f"\n {model_name} Classifier Performance {tag}:")
    print_cv_summary(summary)
    return summary

def print_cv_summary(summary):
    """
    Print the aggregated CV summary (MSE, MAE, R²).
    """
    print(f"Mean MSE:  {summary['mean_mse']:.4f} ± {summary['std_mse']:.4f}")
    print(f"Mean MAE:  {summary['mean_mae']:.4f} ± {summary['std_mae']:.4f}")
    print(f"Mean R²:   {summary['mean_r2']:.4f} ± {summary['std_r2']:.4f}")



In [ ]:
df = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/deconfounded_but_age/aparc.thickness_aseg.volume_aparc.volume.csv")
label_df = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/age_label/all_ages_all_ids_healthy.csv")
n_splits = 5

merged_df = pd.merge(df, label_df, on='ID', how='inner')
merged_df.dropna(inplace=True)
df_sampled, _ = train_test_split(merged_df, train_size=10000, stratify=merged_df["label_age_group"], random_state=42)
df_sampled["label_Age"].value_counts()

y = df_sampled["label_Age"]
col_to_drop = [col for col in label_df.columns]
X = df_sampled.drop(col_to_drop, axis=1)

In [ ]:
merged_df["label_age_group"].value_counts().sort_values(ascending=False)

merged_df["label_age_group"].value_counts()

In [ ]:
import pandas as pd
from sklearn.utils import resample

# Get class distribution
class_counts = merged_df["label_age_group"].value_counts()

# Step 1: Retain all samples from the 3 smallest classes
smallest_classes = class_counts.nsmallest(3).index
df_smallest = merged_df[merged_df["label_age_group"].isin(smallest_classes)]

# Step 2: Calculate remaining samples needed to reach 10,000
remaining_size = 12500- len(df_smallest)

# Step 3: Select the remaining larger classes
remaining_classes = class_counts.nlargest(len(class_counts) - 3).index
df_remaining = merged_df[merged_df["label_age_group"].isin(remaining_classes)]

# Step 4: Determine the number of samples to take per remaining class proportionally
samples_per_class = remaining_size // len(remaining_classes)

# Downsample the remaining classes to fill the dataset
balanced_majority = df_remaining.groupby("label_age_group").apply(
    lambda x: x.sample(n=min(len(x), samples_per_class), random_state=42)
).reset_index(drop=True)

# Combine both parts to form the final balanced dataset
balanced_df = pd.concat([df_smallest, balanced_majority])

# Verify the final class distribution
print(balanced_df["label_age_group"].value_counts())
print("Total samples:", len(balanced_df))


In [ ]:
col_to_drop

In [ ]:
df_control = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/01_Validation_data_set/00_data/final_folder/aparc.thickness_aparc.volume_aseg.volume.csv")
label_df_control = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/01_Validation_data_set/00_data/final_folder/aparc.thickness_aparc.volume_aseg.volume_label.csv")

label_df_control = label_df_control[['ID', 'label_Age']]
df_control = df_control[df.columns]
merged_df_control = pd.merge(df_control, label_df_control, on='ID', how='inner')
merged_df_control.dropna(inplace=True)

X_control = merged_df_control.drop(["ID", "label_Age"], axis=1)
y_control = merged_df_control["label_Age"]

merged_df_control["label_Age"].value_counts()


In [ ]:
#make sure that the order of columns is the same
X = X[X_control.columns]
#check len of X and y
print(len(X), len(y))
print(len(X_control), len(y_control))
#columns number
print(X.shape[1], X_control.shape[1])

for col in X.columns:
    if col not in X_control.columns:
        print(col)



In [ ]:
def feature_extraction_with_Pearson(X, X_val, X_control, y, threshold=0.6, df_columns=None):
    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X)
        if df_columns is not None:
            X.columns = df_columns
    if isinstance(X_val, np.ndarray):
        X_val = pd.DataFrame(X_val)
        if df_columns is not None:
            X_val.columns = df_columns
    if isinstance(X_control, np.ndarray):
        X_control = pd.DataFrame(X_control)
        if df_columns is not None:
            X_control.columns = df_columns
    correlation_matrix = X.corr().abs()
    upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    X = X.drop(columns=to_drop)
    X_val = X_val.drop(columns=to_drop)
    X_control = X_control.drop(columns=to_drop)
    return X.to_numpy(), X_val.to_numpy(), X_control.to_numpy()

def feature_extration_with_PCA(X, X_val, X_control, n_components):
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    X_val_pca = pca.transform(X_val)
    X_control_pca = pca.transform(X_control)
    return X_pca, X_val_pca, X_control_pca

def feature_extration_with_BE(X, X_val, X_control, y, significance_level=0.05, df_columns=None):
    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X)
        if df_columns is not None:
            X.columns = df_columns
    if isinstance(X_val, np.ndarray):
        X_val = pd.DataFrame(X_val)
        if df_columns is not None:
            X_val.columns = df_columns
    if isinstance(X_control, np.ndarray):
        X_control = pd.DataFrame(X_control)
        if df_columns is not None:
            X_control.columns = df_columns
    # Add constant for intercept
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    X = sm.add_constant(X)

    while True:
        # Fit the OLS model
        model = sm.OLS(y, X).fit()
        
        # Get the p-values for each feature
        p_values = model.pvalues
        
        # Find the feature with the highest p-value
        max_p_value = p_values.max()
        
        if max_p_value > significance_level:
            # Remove the feature with the highest p-value
            feature_to_remove = p_values.idxmax()
            print(f"Removing {feature_to_remove} with p-value {max_p_value:.4f}")
            X = X.drop(columns=[feature_to_remove])
            X_val = X_val.drop(columns=[feature_to_remove])
            X_control = X_control.drop(columns=[feature_to_remove])
        else:
            break
        print("Final Feature lengthe: ", len(X.columns))
    # Return the final selected feature set (excluding the intercept)
    return X.drop(columns=['const']).to_numpy(), X_val.to_numpy(), X_control.to_numpy()

In [ ]:
###############################################################################
# K-Fold Cross-Validation Setup
###############################################################################
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

###############################################################################
# Training & Evaluation
###############################################################################
mlp_results = []
lgb_results = []
tabpfn_results = []
random_results = []

mlp_results_eval = []
lgb_results_eval = []
tabpfn_results_eval = []
result_dict = {}

model_dict = {}
best_mse_mlp = float('inf')
best_mse_lgb = float('inf')
best_mse_tab = float('inf')
deconfounding_strategies = ["BE", "Correlation", "PCA" "Nothing"]
for deconfounding_strategy in deconfounding_strategies:
    print(f"\n=== Deconfounding Strategy: {deconfounding_strategy} ===")
    for fold, (train_index, val_index) in enumerate(kf.split(X), 1):
        print(f"\n=== Fold {fold} ===")
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # Scale features
        df_columns = X.columns
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled   = scaler.transform(X_val)
        X_control_scaled = scaler.fit_transform(X_control)

        if deconfounding_strategy == "BE":
            X_train_scaled, X_val_scaled, X_control_scaled = feature_extration_with_BE(X_train_scaled, X_val_scaled, X_control_scaled, y_train, df_columns=df_columns)
        elif deconfounding_strategy == "PCA":
            X_train_scaled, X_val_scaled, X_control_scaled= feature_extration_with_PCA(X_train_scaled, X_val_scaled, X_control_scaled,  n_components=50)
        elif deconfounding_strategy == "Correlation":
            X_train_scaled, X_val_scaled, X_control_scaled = feature_extraction_with_Pearson(X_train_scaled, X_val_scaled, X_control_scaled, y_train, threshold=0.6, df_columns=df_columns)

        # -------------------------------
        # Random Baseline
        # -------------------------------
        # We'll generate random predictions from a normal distribution 
        # matching the train target's mean and std
        random_predictions = np.random.normal(loc=y_train.mean(), scale=y_train.std(), size=len(y_val))
        random_perf = evaluate_regression_performance(y_val, random_predictions)
        print("Random Baseline Performance:")
        print_regression_performance(random_perf)
        random_results.append(random_perf)
        
        # -------------------------------
        # MLP
        # -------------------------------
        # KerasRegressor or direct model
        mlp_model = create_mlp_model(input_shape=X_train_scaled.shape[1])
        mlp_model.fit(X_train_scaled, y_train, 
                    epochs=10, 
                    batch_size=32,
                    verbose=0)
        
        y_pred_mlp = mlp_model.predict(X_val_scaled).ravel()  # ensure shape (n,)
        mlp_perf = evaluate_regression_performance(y_val, y_pred_mlp)
        print("\nMLP Performance on Validation:")
        print_regression_performance(mlp_perf)
        mlp_results.append(mlp_perf)
        
        # Evaluate on control data
        y_pred_mlp_ctrl = mlp_model.predict(X_control_scaled).ravel()
        mlp_perf_ctrl = evaluate_regression_performance(y_control, y_pred_mlp_ctrl)
        print("MLP Performance on Control:")
        print_regression_performance(mlp_perf_ctrl)
        mlp_results_eval.append(mlp_perf_ctrl)
        
        # Keep best MLP model based on MSE
        if mlp_perf['mse'] < best_mse_mlp:
            best_mse_mlp = mlp_perf['mse']
            model_dict["mlp"] = mlp_model
        
        # Clean up
        clean_up_cuda(mlp_model)

        # -------------------------------
        # (Hypothetical) TabPFN Regressor
        # -------------------------------
        # NOTE: If TabPFNClassifier is the only option, you must skip or replace this.
        try:
            tabclf = TabPFNRegressor()  # Ideally TabPFNRegressor() if available
            tabclf.fit(X_train_scaled, y_train)
            y_pred_tab = tabclf.predict(X_val_scaled)  # For regression, this should be continuous
            tab_perf = evaluate_regression_performance(y_val, y_pred_tab)
            print("\nTabPFN Regressor Performance on Validation:")
            print_regression_performance(tab_perf)
            tabpfn_results.append(tab_perf)
            
            # Evaluate on control data
            y_pred_tab_ctrl = tabclf.predict(X_control_scaled)
            tab_perf_ctrl = evaluate_regression_performance(y_control, y_pred_tab_ctrl)
            print("TabPFN Regressor Performance on Control:")
            print_regression_performance(tab_perf_ctrl)
            tabpfn_results_eval.append(tab_perf_ctrl)
            
            if tab_perf['mse'] < best_mse_tab:
                best_mse_tab = tab_perf['mse']
                model_dict["tabpfn"] = tabclf
            
            clean_up_cuda(tabclf)
        except Exception as e:
            print("TabPFN Regressor not available or failed. Skipping...")
            print(e)
        
        # -------------------------------
        # LightGBM
        # -------------------------------
        lgb_train = lgb.Dataset(X_train_scaled, label=y_train)
        lgb_eval  = lgb.Dataset(X_val_scaled,   label=y_val, reference=lgb_train)
        
        lgb_params = {
            'objective': 'regression',
            'metric': 'rmse',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'seed': 42
        }
        
        lgbclf = lgb.train(
            params=lgb_params, 
            train_set=lgb_train, 
            valid_sets=[lgb_train, lgb_eval], 
            num_boost_round=1000
        )
        
        y_pred_lgb = lgbclf.predict(X_val_scaled)
        lgb_perf = evaluate_regression_performance(y_val, y_pred_lgb)
        print("\nLightGBM Performance on Validation:")
        print_regression_performance(lgb_perf)
        lgb_results.append(lgb_perf)
        
        # Evaluate on control data
        y_pred_lgb_ctrl = lgbclf.predict(X_control_scaled)
        lgb_perf_ctrl = evaluate_regression_performance(y_control, y_pred_lgb_ctrl)
        print("LightGBM Performance on Control:")
        print_regression_performance(lgb_perf_ctrl)
        lgb_results_eval.append(lgb_perf_ctrl)
        
        if lgb_perf['mse'] < best_mse_lgb:
            best_mse_lgb = lgb_perf['mse']
            model_dict["lgb"] = lgbclf
        
        clean_up_cuda(lgbclf)

        random_summary = aggregate_cv_metrics_and_print(random_results, "Random")
        tabpfn_summary = aggregate_cv_metrics_and_print(tabpfn_results, "TabPFN")
        lgb_summary = aggregate_cv_metrics_and_print(lgb_results, "LGBM")
        mlp_summary = aggregate_cv_metrics_and_print(mlp_results, "MLP")

        tabpfn_eval_summary = aggregate_cv_metrics_and_print(tabpfn_results_eval, "TabPFN", "Control")
        lgb_eval_summary = aggregate_cv_metrics_and_print(lgb_results_eval, "LGBM", "Control")
        mlp_eval_summary = aggregate_cv_metrics_and_print(mlp_results_eval, "MLP", "Control")




    result_dict[deconfounding_strategy] = {
        "TabPFN": {
                "results": tabpfn_summary,
                "results_eval": tabpfn_eval_summary,
                "cv_results": tabpfn_results,
                "cv_results_eval": tabpfn_results_eval
        },
        "LGBM": {
                "results": lgb_summary,
                "results_eval": lgb_eval_summary,
                "cv_results": lgb_results,
                "cv_results_eval": lgb_results_eval
        },
        "Random": {
                "results": random_summary,
                "cv_results": random_results
        },
        "MLP": {
                "results": mlp_summary,
                "results_eval": mlp_eval_summary,
                "cv_results": mlp_results,
                "cv_results_eval": mlp_results_eval
        }
    }



In [ ]:
result_dict

In [ ]:
for result, models in result_dict.items():
    print(f"\n=== Deconfounding Strategy: {result} ===")
    for model, results in models.items():
        print(f"  {model} - Results: {results['results']}")
        if 'results_eval' in results:
            print(f"  {model} - Evaluation Results: {results['results_eval']}")

In [ ]:

###############################################################################
# Example: Load a saved model & evaluate on control data
###############################################################################
# If you have a saved regression model:
"""
import pickle
save_dir = "../98_models/"
with open(os.path.join(save_dir, "best_regressor.pkl"), "rb") as f:
    loaded_model = pickle.load(f)
    # For example, if it's a LightGBM model, you can just do:
    y_pred_control = loaded_model.predict(X_control_scaled)
    performance_control = evaluate_regression_performance(y_control, y_pred_control)
    print("\nLoaded Model Performance on Control Data:")
    print_regression_performance(performance_control)
"""
